# 通过将数据进行预处理之后，我们就需要来构建我们的数据检索系统
步骤：
1. 选择检索方式（关键词检索、向量检索、图检索、混合检索）
2. 选择检索数据库（Faiss、Annoy、BM25）
3. 选择模型（embeding模型+rerank模型+llm模型）
4. 调整检索参数
   

什么场景下使用哪种检索方式？
1. 关键词检索：搜索之前已经能够准确知道检索内容，且更适合查询数字id，例如查询文件中身份证是xxx的内容
2. 向量检索：搜索之前只知道大概是什么意思，但是不了解具体官方叫法，例如：有时候需要找盖章流程，但是不知道具体叫法，就可以通过向量检索最终找到用印申请流程
3. 图检索：在有大量复杂关系的长文本中搜索或者需要概括各种关系流程，例如：搜索整本小说的任务故事线，或者搜索整本小说中的人物关系（这种检索对于本质上是以部分内容回答的向量检索来说，效果会更好，更全面）
4. 混合检索：如果以上三种需求都有，则可以考虑混合检索多路找回后进行结果融合


## 关键词检索



#### 环境准备

In [ ]:
## 建议使用Elasticsearch，langchain自带的bm25效果很差，也不是传统意义的关键词检索，感兴趣的可以查询一下原理
docker pull docker.elastic.co/elasticsearch/elasticsearch:8.15.3
docker network create elastic
## 可以把端口和密码换一下
docker run -d --name es01 --net elastic -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=true" -e "ELASTIC_PASSWORD=your_password" docker.elastic.co/elasticsearch/elasticsearch:8.15.3
## 测试一下是否可以连接
curl -X GET "http://localhost:9200"



In [4]:
!pip install elasticsearch pandas python-docx --quiet --upgrade

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple


 如果你想要深入学习Elasticsearch，可以参考以下代码和官方文档
 
 [参考代码][1] 
 
 [官方文档][2]

[1]: https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/
[2]: https://elasticsearch-py.readthedocs.io/en/v8.16.0/interactive.html

### 代码样例

In [ ]:
from elasticsearch import Elasticsearch
import urllib3
import logging
import warnings

warnings.filterwarnings("ignore")

# 禁用不安全的 HTTPS 警告（仅用于测试，生产环境应该使用 HTTPS）
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 设置日志级别以查看更多信息
logging.basicConfig(level=logging.INFO)

# 初始化 Elasticsearch
es = Elasticsearch(
    hosts=['http://localhost:39200'],
    basic_auth=('elastic', 'Xieyu120807'),
    verify_certs=False,  # 仅用于测试，生产环境应该验证证书
    request_timeout=30
)
index_name = "rag_teach"

# ## 需要把数据转换为json格式
# ### txt文件
def index_txt_file(file_path, index_name):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    document = {
        "title": file_path.split("/")[-1],
        "content": content
    }
    es.index(index=index_name, document=document)

# ### csv文件
# def index_csv_file(file_path, index_name):
#     df = pd.read_csv(file_path)
#     actions = [
#         {
#             "_index": index_name,
#             "_source": row.to_dict()
#         }
#         for _, row in df.iterrows()
#     ]
    
#     success, _ = bulk(es, actions)
#     print(f"已成功索引 {success} 条记录")

# ### docx文件
# def index_word_document(file_path, index_name):
#     doc = Document(file_path)
#     full_text = []
#     for para in doc.paragraphs:
#         full_text.append(para.text)
    
#     document = {
#         "title": file_path.split("/")[-1],
#         "content": "\n".join(full_text)
#     }
    
#     es.index(index=index_name, document=document)
#     print(f"已索引Word文档: {file_path}")


# 导入数据
index_txt_file(file_path="/data/xieyu/Teaching/RAG/RAG的道与术.txt", index_name="rag_teach")

keyword = "背景"
 
### 直接的关键词检索
query = {
    "query": {
        "match": {
            "content": keyword
        }
    }
}

results = es.search(index=index_name, body=query)
results

### 多种关键词查询方式

In [ ]:
# 导入多维测试数据
import pandas as pd
from elasticsearch import Elasticsearch, helpers
from datetime import datetime

# 连接到Elasticsearch
es = Elasticsearch("http://localhost:39200", basic_auth=('elastic', 'Xieyu120807'), verify_certs=False)

# 确保索引存在
index_name = "book_index"
if not es.indices.exists(index=index_name):
    print(f"索引 {index_name} 不存在，正在创建并导入示例数据。")
    
    # 创建示例数据
    sample_books = [
        {
            "title": "Python编程指南",
            "authors": ["John Smith", "Jane Doe"],
            "publisher": "技术出版社",
            "publish_date": "2023-01-15",
            "summary": "这是一本全面的Python编程指南，适合初学者和中级程序员。",
            "num_reviews": 120
        },
        {
            "title": "JavaScript高级教程",
            "authors": ["Mike Johnson"],
            "publisher": "Addison-Wesley",
            "publish_date": "2022-11-30",
            "summary": "深入探讨JavaScript的高级特性和最佳实践。",
            "num_reviews": 85
        },
        {
            "title": "数据科学实战",
            "authors": ["Sarah Lee", "Tom Brown"],
            "publisher": "数据出版集团",
            "publish_date": "2023-03-22",
            "summary": "通过实际项目学习数据科学，包括Python和R语言的应用。",
            "num_reviews": 56
        }
    ]
    
    # 导入示例数据
    for book in sample_books:
        es.index(index=index_name, document=book)
    
    print("示例数据已成功导入。")
else:
    print(f"索引 {index_name} 已存在，准备进行查询示例。")

In [12]:
## 多种查询方式示例
# 定义美化输出函数
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("您的搜索没有返回结果。")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            publication_date = hit["_source"]["publish_date"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            summary = hit["_source"]["summary"]
            publisher = hit["_source"]["publisher"]
            num_reviews = hit["_source"]["num_reviews"]
            authors = ", ".join(hit["_source"]["authors"])
            pretty_output = f"\nID: {id}\n出版日期: {publication_date}\n标题: {title}\n摘要: {summary}\n出版社: {publisher}\n评论数: {num_reviews}\n作者: {authors}\n相关度得分: {score}"
            print(pretty_output)

# 1. 简单的关键词匹配查询
def simple_keyword_query(keyword):
    query = {
        "query": {
            "match": {
                "summary": keyword
            }
        }
    }
    results = es.search(index=index_name, body=query)
    print(f"关键词 '{keyword}' 的简单匹配查询结果:")
    print("说明：这种查询方式会在summary字段中搜索包含指定关键词的文档。")
    pretty_response(results)

# 2. 多字段查询
def multi_field_query(keyword):
    query = {
        "query": {
            "multi_match": {
                "query": keyword,
                "fields": ["summary", "title"]
            }
        }
    }
    results = es.search(index=index_name, body=query)
    print(f"关键词 '{keyword}' 的多字段查询结果:")
    print("说明：这种查询方式会在summary和title字段中搜索包含指定关键词的文档。")
    pretty_response(results)

# 3. 带权重的多字段查询
def weighted_multi_field_query(keyword):
    query = {
        "query": {
            "multi_match": {
                "query": keyword,
                "fields": ["summary", "title^3"]
            }
        }
    }
    results = es.search(index=index_name, body=query)
    print(f"关键词 '{keyword}' 的带权重多字段查询结果:")
    print("说明：这种查询方式会在summary和title字段中搜索，但title字段的权重是summary的3倍。")
    pretty_response(results)

# 4. 精确匹配查询
def term_query(publisher):
    query = {
        "query": {
            "term": {
                "publisher.keyword": publisher
            }
        }
    }
    results = es.search(index=index_name, body=query)
    print(f"出版社 '{publisher}' 的精确匹配查询结果:")
    print("说明：这种查询方式要求publisher字段完全匹配指定的值，不进行分词。")
    pretty_response(results)

# 执行查询示例
# 执行查询示例
simple_keyword_query("Python")
multi_field_query("数据科学")
weighted_multi_field_query("编程")
term_query("技术出版社")

# 解释每个查询的结果
print("\n简单关键词查询解释:")
print("这个查询会在summary字段中搜索包含'Python'的文档。")
print("预期会匹配到'Python编程指南'这本书。")
print("--------------------------------")
print("\n多字段查询解释:")
print("这个查询会在summary和title字段中搜索包含'数据科学'的文档。")
print("预期会匹配到'数据科学实战'这本书。")
print("--------------------------------")
print("\n带权重的多字段查询解释:")
print("这个查询会在summary和title字段中搜索包含'编程'的文档，但title字段的权重更高。")
print("预期可能会匹配到'Python编程指南'，因为'编程'在title中出现。")
print("--------------------------------")
print("\n精确匹配查询解释:")
print("这个查询要求publisher字段完全匹配'技术出版社'。")
print("预期会匹配到'Python编程指南'这本书。")
print("--------------------------------")
# 注意：实际结果可能会因为Elasticsearch的评分机制而略有不同

# 注意：这些查询示例基于我们创建的示例数据。
# 在实际应用中，您可能需要根据实际的数据结构和需求调整查询。


INFO:elastic_transport.transport:POST http://localhost:39200/book_index/_search [status:200 duration:0.008s]
INFO:elastic_transport.transport:POST http://localhost:39200/book_index/_search [status:200 duration:0.007s]
INFO:elastic_transport.transport:POST http://localhost:39200/book_index/_search [status:200 duration:0.006s]
INFO:elastic_transport.transport:POST http://localhost:39200/book_index/_search [status:200 duration:0.005s]


关键词 'Python' 的简单匹配查询结果:
说明：这种查询方式会在summary字段中搜索包含指定关键词的文档。

ID: ufCfOZMBE5wK0lLCF5Q1
出版日期: 2023-03-22
标题: 数据科学实战
摘要: 通过实际项目学习数据科学，包括Python和R语言的应用。
出版社: 数据出版集团
评论数: 56
作者: Sarah Lee, Tom Brown
相关度得分: 0.45153183

ID: t_CfOZMBE5wK0lLCFpT2
出版日期: 2023-01-15
标题: Python编程指南
摘要: 这是一本全面的Python编程指南，适合初学者和中级程序员。
出版社: 技术出版社
评论数: 120
作者: John Smith, Jane Doe
相关度得分: 0.44283003
关键词 '数据科学' 的多字段查询结果:
说明：这种查询方式会在summary和title字段中搜索包含指定关键词的文档。

ID: ufCfOZMBE5wK0lLCF5Q1
出版日期: 2023-03-22
标题: 数据科学实战
摘要: 通过实际项目学习数据科学，包括Python和R语言的应用。
出版社: 数据出版集团
评论数: 56
作者: Sarah Lee, Tom Brown
相关度得分: 3.7324529

ID: t_CfOZMBE5wK0lLCFpT2
出版日期: 2023-01-15
标题: Python编程指南
摘要: 这是一本全面的Python编程指南，适合初学者和中级程序员。
出版社: 技术出版社
评论数: 120
作者: John Smith, Jane Doe
相关度得分: 0.44283003
关键词 '编程' 的带权重多字段查询结果:
说明：这种查询方式会在summary和title字段中搜索，但title字段的权重是summary的3倍。

ID: t_CfOZMBE5wK0lLCFpT2
出版日期: 2023-01-15
标题: Python编程指南
摘要: 这是一本全面的Python编程指南，适合初学者和中级程序员。
出版社: 技术出版社
评论数: 120
作者: John Smith, Jane Doe
相关度得分: 4.4667044

ID: uPCfOZMBE5wK0lLCF5Qr
出版日期: 202

## 向量检索

In [ ]:
"""
说明：向量检索主要分为四个步骤：
1. 把文本内容分块（如果非文本内容请参考上一个章节）
2. 把分块后的文本内容转换为向量 
3. 把向量保存到数据库中（Faiss、Annoy、Chroma）
4. 检索时，把查询内容转换为向量，在数据库中查找最相似的向量，返回结果
"""

<img src="/data/xieyu/Teaching/RAG/imgs/embeding_process.png" alt="向量检索流程图" width="30%" style="display: block; margin: auto;">
<p style="text-align: center;">上图展示了向量检索的基本流程，包括文本分块、向量转换、向量存储和相似度检索等步骤。</p>

#### 文本分块

In [10]:
"""
PS： 由于向量检索需要把文本内容转换为向量存储到向量库进行存储后方便检索，但是向量库检索有维度的限制，维度越大，记录越详细，
但一般模型的维度都是有限制的，然后如果全部文本转换为向量，会导致内容压缩严重，导致后续检索不准
所以需要对文本进行分块，然后对每块文本转换为向量后存储到向量库中，检索时，把查询内容转换为向量后，在向量库中查找最相似的向量，然后返回结果
"""

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载文本
loader = TextLoader("/data/xieyu/Teaching/RAG/RAG的道与术.txt", encoding="utf-8")
documents = loader.load()

# 创建文本分割器
"""
参数讲解：
chunk_size：每块文本的大小
chunk_overlap：每块文本之间的重叠部分（如果一个章节的内容太长，按照chunk_size分块后，可能会让原本的意思断开，所以设置chunk_overlap，则会把保留前后的文本，来保证文本的完整性）
length_function：计算文本长度的函数，默认是len，即字符串长度
"""
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# 分割文本
text_chunks = text_splitter.split_documents(documents)
print(text_chunks)

[Document(metadata={'source': '/data/xieyu/Teaching/RAG/RAG的道与术.txt'}, page_content='RAG的道与术\n目录\n\n目标读者\n想要开始了解并学习 RAG技术的人\n想要利用RAG实现智能知识问答的人\nRAG是什么？\n定义\nRAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术\n背景\nAI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题\n场景\n企业知识库\n客服机器人\n产品自动推荐\nRAG主流方案\n核心步骤\n收集数据\n导入数据\n问题检索\n结果评估\n迭代优化\n用dify进行演示\n主流应用\nlangchain\n上手方便，装包随时可以使用\ndbgpt\n比较通用，api使用方便，原生支持graphRAG\nragflow\n文档处理最方便\nQanything\n检索到的重排序效果最好\nfastgpt\n接入模块易操作\ndify\n结合工作流更方便\n制作对应的表格\nRAG实践问题\n核心目标\n提高AI回复知识库相关问题的准确性\n实践痛点\n提问的问题稍微跟原文换一种说法就回答不上来\n根据自己的理解问问题\n根据自己的记忆关键词问问题\n查询到的内容都不是自己要的内容\n图片表格的知识信息无法查询\n对长文本提问总结性问题总是回答不全\n对于查询不到的问题进行编造强行回答\n知识库特别大，难以完全测试来保证准确度\n优化步骤\n知道放什么知识？\n知道怎么放知识？\n知道怎么查知识？\n知道查的对不对？\n实践拆解\n知道放什么知识？\n范围\n尽量只放入跟后续提问相关的内容，无效的内容越多，查询准确率越低\n质量\n内容要求要准确严谨，互相矛盾，避免大模型产生混乱\n类型\n本质上大模型只能够回答文本的内容，所以导入的文档应该尽量转化成普通文本模式\n知识到怎么放知识\n知识类型\n纯文本导入\n带图片文本导入\n带表格文本导入\n导入数据库\nrelation-db\nstr\ngraph-db\nnode\nvector-db\n转化成

#### 向量导入 

In [ ]:
"""
PS：向量导入首先要选择一个embeding模型，然后对文本分块转换为向量后，保存到向量库中
"""

##### 如何选择embeding模型

如果你是第一次使用RAG，那非常建议你直接查看MTEB选择排名最好的模型，然后直接使用，但是要根据自己的预料的语言进行选择，中文就选zn的模型，英文就选en的模型
<img src="/data/xieyu/Teaching/RAG/imgs/mteb_benchmark.png" alt="MTEB排行榜" width="30%" style="display: block; margin: auto;">
<p style="text-align: center;">上图展示了<a href="https://huggingface.co/spaces/mteb/leaderboard" target="_blank">MTEB排行榜</a>，可以参考选择模型</p>

如果你是一个老鸟，那考虑embeding模型主要考虑以下几个因素：
1. <font color="white">模型性能（准确率、召回率、F1值）</font>
2. <font color="white">模型速度</font>
3. <font color="white">支持向量维度大小</font>
4. <font color="white">模型大小</font>

这些核心指标大部分可以从榜单中找到，但是具体还是应该<font color="red">结合自己的数据集进行测试</font>，因为不同的数据集的效果差异很大，一定程度上需要对mteb祛魅，<font color="red">基本选评分最高的不会错</font>，但是追求最好效果就需要测试，建议是用<font color="red">xinference去一键部署，效率最高</font>

<img src="/data/xieyu/Teaching/RAG/imgs/xinference.png" alt="xinference" width="30%" style="display: block; margin: auto;">
<p style="text-align: center;"><a href="https://github.com/xorbitsai/inference" target="_blank">xinference平台</a></p>

In [ ]:
"""
使用方法：
1. 使用本地下载的模型
2. 使用xinference部署模型
3. 使用openai接口类型的api
"""

"""
下载模型最好配置好cache_dir，这样模型会下载到cache_dir中，方便后续进行统一管理，如果保持默认值会放在一个临时目录中，比较难记得，后续多个项目要使用的时候不方便
"""
# from modelscope.hub.snapshot_download import snapshot_download
# model_dir = snapshot_download('BAAI/bge-m3', cache_dir='/data/lilk/RAG/models/embeding', revision='v1.5')
# 使用huggingface下载模型
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="ibrahimhamamci/CT-RATE", repo_type="dataset", cache_dir="/data/lilk/RAG/models/embeding")



In [23]:
# openai调用, 更加方便快捷，也不需要显卡，但很多国内的用户没有openai的账号，所以推荐使用硅基云的embeding模型，也支持openai的api格式的调用
from langchain_openai import OpenAIEmbeddings
embeding_model = OpenAIEmbeddings(model="BAAI/bge-m3",api_key="sk-nmyyoncsmaagafmvjmbpyaxbeewtwqaiycitmhtomjzlwbsw",base_url="https://api.siliconflow.cn/v1")

In [6]:
!pip install -qU langchain_huggingface --upgrade

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# 本地调用
embeding_model_path = "/data/lilk/RAG/models/embeding/bge-m3"
# 配置运行的参数
embeding_model_kwargs = {"device": "cuda"}
embeding_encode_kwargs = {"normalize_embeddings": True}
embeding_model = HuggingFaceEmbeddings(model_name = embeding_model_path,
                                       model_kwargs=embeding_model_kwargs,
                                       encode_kwargs=embeding_encode_kwargs)


/data/lilk/RAG/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [21]:
# xinference调用
from langchain_community.embeddings import XinferenceEmbeddings
embeding_model = XinferenceEmbeddings(
    server_url="http://0.0.0.0:19997",  # Xinference服务的URL
    model_uid="bge-m3"  # 从步骤2中获取的模型UID
)

##### 如何选择一个向量库？

| 向量数据库     | 特点                                                         |
|--------------|--------------------------------------------------------------|
| Milvus       | - 开源向量数据库<br>- 适合大规模向量数据存储和检索<br>- 支持多种ANN算法 |
| Faiss        | - 由Meta开发的高效相似性搜索和密集向量聚类库<br>- 支持大规模向量集搜索 |
| Weaviate     | - 开源向量数据库<br>- 支持水平和垂直扩展<br>- 适合数十亿数据对象 |
| Pinecone     | - 专为机器学习设计的向量数据库<br>- 快速、可扩展<br>- 支持实时更新 |
| Qdrant       | - 矢量相似度搜索引擎和数据库<br>- 支持分布式部署和水平扩展         |
| Chroma       | - 轻量级、易用的向量数据库<br>- 适合小型到中型数据集             |
| Elasticsearch| - 分布式搜索和分析引擎<br>- 从7.10版本开始支持向量字段           |

向量数据库的推荐其实也是要依赖数据的特点，但是刚开始实现一个项目的时候，往往不太清楚数据的特点，所以**选择更容易调用的比选择更适合的更重要**，构建完整闭环后再回来优化也不迟，这里选择Faiss数据库进行演示（<font color="red">封装得比较好，该有的功能也都有</font>）

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
#  用刚刚的embeding模型和分块后的文本转换的向量来初始化向量库
"""
参数解析：
embedding：引用刚刚初始化的embeding模型
documents：分块后的文本
docstore：用来存储导入的分块：InMemoryDocstore()是用内存来存储，但是如果数据量太大，可以使用ElasticSearchDocStore，或者SQLDocStore
```
    from langchain.docstore import SQLDocStore
    docstore = SQLDocStore(
        db_url="sqlite:///documents.db"
    )
```
index_to_docstore_id：文档存储的id，方便后面检索的时候，可以知道是哪个文档
"""
vector_store = FAISS.from_documents(embedding=embeding_model,documents=text_chunks,docstore=InMemoryDocstore(),index_to_docstore_id={})
# 检索出结果
vector_store.similarity_search("RAG是什么？")


[Document(metadata={'source': '/data/xieyu/Teaching/RAG/RAG的道与术.txt'}, page_content='RAG的道与术\n目录\n\n目标读者\n想要开始了解并学习 RAG技术的人\n想要利用RAG实现智能知识问答的人\nRAG是什么？\n定义\nRAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术\n背景\nAI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题\n场景\n企业知识库\n客服机器人\n产品自动推荐\nRAG主流方案\n核心步骤\n收集数据\n导入数据\n问题检索\n结果评估\n迭代优化\n用dify进行演示\n主流应用\nlangchain\n上手方便，装包随时可以使用\ndbgpt\n比较通用，api使用方便，原生支持graphRAG\nragflow\n文档处理最方便\nQanything\n检索到的重排序效果最好\nfastgpt\n接入模块易操作\ndify\n结合工作流更方便\n制作对应的表格\nRAG实践问题\n核心目标\n提高AI回复知识库相关问题的准确性\n实践痛点\n提问的问题稍微跟原文换一种说法就回答不上来\n根据自己的理解问问题\n根据自己的记忆关键词问问题\n查询到的内容都不是自己要的内容\n图片表格的知识信息无法查询\n对长文本提问总结性问题总是回答不全\n对于查询不到的问题进行编造强行回答\n知识库特别大，难以完全测试来保证准确度\n优化步骤\n知道放什么知识？\n知道怎么放知识？\n知道怎么查知识？\n知道查的对不对？\n实践拆解\n知道放什么知识？\n范围\n尽量只放入跟后续提问相关的内容，无效的内容越多，查询准确率越低\n质量\n内容要求要准确严谨，互相矛盾，避免大模型产生混乱\n类型\n本质上大模型只能够回答文本的内容，所以导入的文档应该尽量转化成普通文本模式\n知识到怎么放知识\n知识类型\n纯文本导入\n带图片文本导入\n带表格文本导入\n导入数据库\nrelation-db\nstr\ngraph-db\nnode\nvector-db\n转化成

优化建议：我们使用RAG的过程中肯定是需要长期优化增删改查这个知识库的，所以我们需要把刚刚的初次生成的向量库保存下来，方便后面进行维护，下面我们用一个从头开始初始化向量库的代码来实践看看，而且如果我们是制作一个RAG的api，那不可能每次启动服务的时候都用embeding去初始化相同的文档去做嵌入，如果文本很多的情况下，会很耗费资源

In [11]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# 初始化faiss索引
"""
为什么需要初始化索引？
1. 类似于我们要创建一个表的时候要确定多少列，这样在后续查询的时候才知道我们只能查到多少列是有效数据
2. 用helloworld来生成索引的维度，不是很重要，因为helloworld的维度是固定的，所以用helloworld来生成索引的维度，只是为了获取维度，后续我们用分块后的文本转换的向量来生成索引
"""
index = faiss.IndexFlatL2(len(embeding_model.embed_query("hello world")))

# 初始化向量库
vector_store = FAISS(
    # 引用刚刚初始化的embeding模型
    embedding_function=embeding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
# 向量库的写入方法（其他部分可以查看faiss的文档）
vector_store.add_documents(documents=text_chunks)


# 保存到本地
folder_path = "/data/xieyu/Teaching/RAG/data_retriever/rag_teach"
vector_store.save_local(index_name="rag_teach",folder_path=folder_path)


In [12]:
# 加载本地向量库，并查询
vector_store = FAISS.load_local(index_name="rag_teach",folder_path=folder_path,embeddings=embeding_model,allow_dangerous_deserialization=True )
vector_store.similarity_search("RAG是什么？")

[Document(metadata={'source': '/data/xieyu/Teaching/RAG/RAG的道与术.txt'}, page_content='RAG的道与术\n目录\n\n目标读者\n想要开始了解并学习 RAG技术的人\n想要利用RAG实现智能知识问答的人\nRAG是什么？\n定义\nRAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术\n背景\nAI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题\n场景\n企业知识库\n客服机器人\n产品自动推荐\nRAG主流方案\n核心步骤\n收集数据\n导入数据\n问题检索\n结果评估\n迭代优化\n用dify进行演示\n主流应用\nlangchain\n上手方便，装包随时可以使用\ndbgpt\n比较通用，api使用方便，原生支持graphRAG\nragflow\n文档处理最方便\nQanything\n检索到的重排序效果最好\nfastgpt\n接入模块易操作\ndify\n结合工作流更方便\n制作对应的表格\nRAG实践问题\n核心目标\n提高AI回复知识库相关问题的准确性\n实践痛点\n提问的问题稍微跟原文换一种说法就回答不上来\n根据自己的理解问问题\n根据自己的记忆关键词问问题\n查询到的内容都不是自己要的内容\n图片表格的知识信息无法查询\n对长文本提问总结性问题总是回答不全\n对于查询不到的问题进行编造强行回答\n知识库特别大，难以完全测试来保证准确度\n优化步骤\n知道放什么知识？\n知道怎么放知识？\n知道怎么查知识？\n知道查的对不对？\n实践拆解\n知道放什么知识？\n范围\n尽量只放入跟后续提问相关的内容，无效的内容越多，查询准确率越低\n质量\n内容要求要准确严谨，互相矛盾，避免大模型产生混乱\n类型\n本质上大模型只能够回答文本的内容，所以导入的文档应该尽量转化成普通文本模式\n知识到怎么放知识\n知识类型\n纯文本导入\n带图片文本导入\n带表格文本导入\n导入数据库\nrelation-db\nstr\ngraph-db\nnode\nvector-db\n转化成

#### 知识库查询


刚刚不是已经演示过如何查询了吗？为什么还需要再分一个部分来讲？

随着RAG技术的发展，很多人发现embeding模型在检索的时候，效果并不是很好，所以就有人提出了rerank模型，rerank模型可以理解为在检索后，对检索到的结果进行一个重新的排序，从而提高检索的准确率
那关于rerank模型主要有3个问题:
1. rerank模型是什么？跟embeding模型有什么区别？
    其他rerank模型和embeding模型都是用来计算文本相似度的模型，但是他们的计算算法不同，rerank精准，但是速度慢，embeding速度快，但是效果不如rerank，因此现在基本上是先用embeding模型检索出文档范围，然后再用rerank模型进行排序，提高检索的精度，实现速度和精准度的平衡，如果想要了解他们的具体区别可以了解这位大佬的视频：https://www.bilibili.com/video/BV1r1421R77Y/?spm_id_from=333.337.search-card.all.click&vd_source=8b3e87015d9483289ee62c8c6c603927
2. rerank模型怎么选?
    参考embeding模型的选择
3. rerank模型怎么用？
    api方式或者langchain调用（想见代码）


<img src="/data/xieyu/Teaching/RAG/imgs/embeding+rerank.png" alt="embeding+rerank" width="30%" style="display: block; margin: auto;">

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker

# 初始化retriever+rerank检索器
vector_store = FAISS.load_local(index_name="rag_teach",folder_path=folder_path,embeddings=embeding_model,allow_dangerous_deserialization=True )
rerank_model = HuggingFaceCrossEncoder(model_name="/data/lilk/RAG/models/rerank/bge-reranker-v2-m3")
## 配置embeding模型返回25个，相似度阈值为0.001（有个小细节，相似度阈值的范围如果要控制是0-1，一定要在初始化embeding模型的时候设置normalize_embeddings=True）
retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 25, "score_threshold": 0.001})
## 配置rerank模型，返回10个
compressor = CrossEncoderReranker(model=rerank_model, top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
# 检索
compression_retriever.get_relevant_documents("RAG是什么？")

/data/lilk/RAG/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': '/data/xieyu/Teaching/RAG/RAG的道与术.txt'}, page_content='RAG的道与术\n目录\n\n目标读者\n想要开始了解并学习 RAG技术的人\n想要利用RAG实现智能知识问答的人\nRAG是什么？\n定义\nRAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术\n背景\nAI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题\n场景\n企业知识库\n客服机器人\n产品自动推荐\nRAG主流方案\n核心步骤\n收集数据\n导入数据\n问题检索\n结果评估\n迭代优化\n用dify进行演示\n主流应用\nlangchain\n上手方便，装包随时可以使用\ndbgpt\n比较通用，api使用方便，原生支持graphRAG\nragflow\n文档处理最方便\nQanything\n检索到的重排序效果最好\nfastgpt\n接入模块易操作\ndify\n结合工作流更方便\n制作对应的表格\nRAG实践问题\n核心目标\n提高AI回复知识库相关问题的准确性\n实践痛点\n提问的问题稍微跟原文换一种说法就回答不上来\n根据自己的理解问问题\n根据自己的记忆关键词问问题\n查询到的内容都不是自己要的内容\n图片表格的知识信息无法查询\n对长文本提问总结性问题总是回答不全\n对于查询不到的问题进行编造强行回答\n知识库特别大，难以完全测试来保证准确度\n优化步骤\n知道放什么知识？\n知道怎么放知识？\n知道怎么查知识？\n知道查的对不对？\n实践拆解\n知道放什么知识？\n范围\n尽量只放入跟后续提问相关的内容，无效的内容越多，查询准确率越低\n质量\n内容要求要准确严谨，互相矛盾，避免大模型产生混乱\n类型\n本质上大模型只能够回答文本的内容，所以导入的文档应该尽量转化成普通文本模式\n知识到怎么放知识\n知识类型\n纯文本导入\n带图片文本导入\n带表格文本导入\n导入数据库\nrelation-db\nstr\ngraph-db\nnode\nvector-db\n转化成

## 图检索

##### 为什么需要图检索？

引入一个例子来说明：
有一本书叫《小明与他的100个前女友》，总共10w字，然后如果我想要检索计算小明总共和多少个前女友亲亲过？

【关键词检索】：这种时候会这10w的文章中检索出亲亲，然后把所有出现的亲亲的段落返回，这种情况会把一些跟小明无关的亲亲也返回，导致结果不准确，同时也有可能返回文本太长，导致结果不准确
【向量检索】：向量检索也是查询出有出现亲亲的段落，但是好处就是可以把一些相关的行为，例如接吻的段落也返回，但是缺点就是如果文本量很大，检索的速度会很慢
【图检索】：这种时候会构建一个图谱，图谱中包含小明、前女友、亲亲，然后检索的时候，会根据图谱中小明到前女友的边，然后返回所有小明到前女友的边，这样就可以避免关键词检索带来的问题，同时也可以把一些相关的行为，例如小明的接吻的段落都能精准返回，而且检索的速度也会很快

总结为什么要用图检索？
1. 图检索可以精准返回想要的结果
2. 图检索可以检索出相关的行为
3. 图检索的检索速度非常快

图检索的缺点：
要构建出图谱的关系，其实需要把分块的内容（就是书里面的每个章节）给大模型去分析关系，就会需要耗费比较多的token


##### 如何使用图检索？

常见解决方案是：graphRAG；lightRAG；dbgpt的TugraphRAG，或者ragflow的graphRAG，目前测试下来lightRAG在准确度上和速度上都最强

In [ ]:
## 拉取lightRAG代码
git clone https://github.com/HKUDS/LightRAG
## 安装lightRAG依赖
pip install -r requirements.txt

In [ ]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete, hf_model_complete, hf_embedding, openai_complete_if_cache
from lightrag.utils import EmbeddingFunc
from transformers import AutoModel, AutoTokenizer
import nest_asyncio
nest_asyncio.apply()

# 设置知识库的目录
WORKING_DIR = "./graphRAG"


if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)


# 设置llm模型
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], **kwargs
) -> str:
    return await openai_complete_if_cache(
        "gpt-4o",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key="your key",
        base_url="your base url",
        **kwargs,
    )

embedding_func=EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8000,
    func=lambda texts: hf_embedding(
        texts,
        # 引用刚刚下载的embeding模型
        tokenizer=AutoTokenizer.from_pretrained("/data/lilk/RAG/models/embeding/bge-m3"),
        embed_model=AutoModel.from_pretrained("/data/lilk/RAG/models/embeding/bge-m3")
    )
)

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,  # Use gpt_4o_mini_complete LLM model
    embedding_func=embedding_func,
    # llm_model_func=gpt_4o_complete  # Optionally, use a stronger model
)

import textract

# 把文档转化成txt插入到知识库中
for filename in os.listdir("/data/xieyu/LightRAG/tugraph-doc"):
    filepath = os.path.join("/data/xieyu/LightRAG/tugraph-doc", filename)
    if os.path.isfile(filepath) and filepath.endswith('.pdf'):
        text_content = textract.process(filepath)
        rag.insert(text_content.decode('utf-8'))


不同的检索方式

In [ ]:
"""
a) Naive 搜索（朴素搜索）：
    这是最简单的搜索方法。
    它可能会直接在整个数据集中进行关键词匹配或简单的相似度计算。
    速度可能较快，但准确性可能较低。
b) Local 搜索（局部搜索）：
    这种方法可能会先定位到图中的某个局部区域，然后在该区域内进行搜索。
    适合于当我们大致知道答案可能在哪个区域时使用。
    可以提高搜索效率，但如果初始定位不准确，可能会错过重要信息。
c) Global 搜索（全局搜索）：
    这种方法会在整个图结构中进行搜索。
    相比局部搜索，它可能会更全面，但也可能更耗时。
    适合于需要考虑整个数据集的复杂查询。
d) Hybrid 搜索（混合搜索）：
    这种方法结合了其他搜索策略的优点。
    可能会先进行全局搜索来定位相关区域，然后在这些区域进行更详细的局部搜索。
    通常能在效率和准确性之间取得较好的平衡。
"""

# Perform naive search
print(rag.query("如何查询数据库中现有角色及其相关信息？", param=QueryParam(mode="naive")))

# Perform local search
print(rag.query("这些文档主要的内容是什么？", param=QueryParam(mode="local")))

# Perform global search
print(rag.query("这些文档主要的内容是什么？", param=QueryParam(mode="global")))

# Perform hybrid search
print(rag.query("这些文档主要的内容是什么？", param=QueryParam(mode="hybrid")))